In [ ]:
!pip install openai
!pip install langchain
!pip install langchain_community
!pip install langchain_openai

In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI

# loading from a .env file
# load_dotenv(dotenv_path="/full/path/to/your/.env")

# or 
# if you're on google colab just uncomment below and replace with your openai api key
# os.environ["OPENAI_API_KEY"] = "<your-openai-api-key>"

# Quiz App

- Given a page of a pdf
- Given a concept
- Simple interface
- Quickly generate flexible quizzes
  - Generate any type of questions (True or False, text based questions, multiple choice)
- Feedback on the answers

In [1]:
from openai import OpenAI

def get_response(prompt):
    client = OpenAI()
    response = client.chat.completions.create(model="gpt-3.5-turbo-1106", 
                             messages=
                             [
                                 {"role": "system", "content": "You are a helpful assistant."},
                                 {"role": "user", "content": prompt}   
                             ],
                             temperature=0.0,
                             n = 1
                             )
    return response.choices[0].message.content


prompt = "Create a quiz about the basics of Python programming."
quiz_python = get_response(prompt)
quiz_python

'Sure! Here\'s a short quiz about the basics of Python programming:\n\n1. What is Python?\n   a) A type of snake\n   b) A high-level programming language\n   c) A type of data structure\n\n2. What is the syntax for printing "Hello, World!" in Python?\n   a) print("Hello, World!")\n   b) print "Hello, World!"\n   c) print: "Hello, World!"\n\n3. Which of the following is not a valid variable name in Python?\n   a) my_variable\n   b) 123variable\n   c) _myVariable\n\n4. What is the result of 5 + 3 * 2 in Python?\n   a) 16\n   b) 11\n   c) 13\n\n5. What is the purpose of the "if" statement in Python?\n   a) To define a function\n   b) To loop through a sequence of items\n   c) To execute code based on a condition\n\nAnswers:\n1. b) A high-level programming language\n2. a) print("Hello, World!")\n3. b) 123variable\n4. b) 11\n5. c) To execute code based on a condition'

In [2]:
from IPython.display import Markdown, display

Markdown(quiz_python)

Sure! Here's a short quiz about the basics of Python programming:

1. What is Python?
   a) A type of snake
   b) A high-level programming language
   c) A type of data structure

2. What is the syntax for printing "Hello, World!" in Python?
   a) print("Hello, World!")
   b) print "Hello, World!"
   c) print: "Hello, World!"

3. Which of the following is not a valid variable name in Python?
   a) my_variable
   b) 123variable
   c) _myVariable

4. What is the result of 5 + 3 * 2 in Python?
   a) 16
   b) 11
   c) 13

5. What is the purpose of the "if" statement in Python?
   a) To define a function
   b) To loop through a sequence of items
   c) To execute code based on a condition

Answers:
1. b) A high-level programming language
2. a) print("Hello, World!")
3. b) 123variable
4. b) 11
5. c) To execute code based on a condition

Requirements for the app

1. We have to be specific
   1. We want to specify the number of questions we want
2. We have to be able to feed in contextual information
3. We want to be able to specify the types of questions we want created (T/F, multiple choice, text...)

In [3]:
from langchain.prompts import ChatPromptTemplate

In [4]:
template = """
You are an expert quiz maker for technical fields. Let's think step by step and
create a quiz with {num_questions} {quiz_type} questions about the following concept/content: {quiz_context}.

The format of the quiz could be one of the following:
- Multiple-choice: 
- Questions:
    <Question1>: <a. Answer 1>, <b. Answer 2>, <c. Answer 3>, <d. Answer 4>
    <Question2>: <a. Answer 1>, <b. Answer 2>, <c. Answer 3>, <d. Answer 4>
    ....
- Answers:
    <Answer1>: <a|b|c|d>
    <Answer2>: <a|b|c|d>
    ....
    Example:
    - Questions:
    - 1. What is the time complexity of a binary search tree?
        a. O(n)
        b. O(log n)
        c. O(n^2)
        d. O(1)
    - Answers: 
        1. b
- True-false:
    - Questions:
        <Question1>: <True|False>
        <Question2>: <True|False>
        .....
    - Answers:
        <Answer1>: <True|False>
        <Answer2>: <True|False>
        .....
    Example:
    - Questions:
        - 1. What is a binary search tree?
        - 2. How are binary search trees implemented?
    - Answers:
        - 1. True
        - 2. False
- Open-ended:
- Questions:
    <Question1>: 
    <Question2>:
- Answers:    
    <Answer1>:
    <Answer2>:
Example:
    Questions:
    - 1. What is a binary search tree?
    - 2. How are binary search trees implemented?
    
    - Answers: 
        1. A binary search tree is a data structure that is used to store data in a sorted manner.
        2. Binary search trees are implemented using linked lists.
"""

In [5]:
prompt = ChatPromptTemplate.from_template(template)

In [6]:
prompt.format(num_questions=3, quiz_type="multiple-choice", quiz_context="Data Structures in Python Programming")

"Human: \nYou are an expert quiz maker for technical fields. Let's think step by step and\ncreate a quiz with 3 multiple-choice questions about the following concept/content: Data Structures in Python Programming.\n\nThe format of the quiz could be one of the following:\n- Multiple-choice: \n- Questions:\n    <Question1>: <a. Answer 1>, <b. Answer 2>, <c. Answer 3>, <d. Answer 4>\n    <Question2>: <a. Answer 1>, <b. Answer 2>, <c. Answer 3>, <d. Answer 4>\n    ....\n- Answers:\n    <Answer1>: <a|b|c|d>\n    <Answer2>: <a|b|c|d>\n    ....\n    Example:\n    - Questions:\n    - 1. What is the time complexity of a binary search tree?\n        a. O(n)\n        b. O(log n)\n        c. O(n^2)\n        d. O(1)\n    - Answers: \n        1. b\n- True-false:\n    - Questions:\n        <Question1>: <True|False>\n        <Question2>: <True|False>\n        .....\n    - Answers:\n        <Answer1>: <True|False>\n        <Answer2>: <True|False>\n        .....\n    Example:\n    - Questions:\n        

In [7]:
from langchain_openai import ChatOpenAI
from langchain.schema import StrOutputParser
import re

In [8]:
chain = prompt | ChatOpenAI(temperature=0.0) | StrOutputParser()

In [9]:
quiz_response = chain.invoke({"num_questions":3, "quiz_type":"multiple-choice", "quiz_context":"Data Structures in Python Programming",})

In [10]:
Markdown(quiz_response)

- Multiple-choice:
- Questions:
    1. What is a data structure in Python programming?
        a. A way to store and organize data
        b. A programming language
        c. A type of loop
        d. A sorting algorithm
    2. Which of the following is not a built-in data structure in Python?
        a. List
        b. Tuple
        c. Stack
        d. Tree
    3. What is the time complexity of accessing an element in a Python dictionary?
        a. O(1)
        b. O(log n)
        c. O(n)
        d. O(n^2)
- Answers:
    1. a
    2. d
    3. a

In [11]:
quiz_response = chain.invoke({"num_questions": 4, "quiz_type": "True-false", "quiz_context": "Python programming"})

In [12]:
Markdown(quiz_response)

- True-false:
    - Questions:
        1. Python is a statically typed language. 
        2. Python is an interpreted language.
        3. Python supports multiple inheritance.
        4. Python uses curly braces to define code blocks.
    - Answers:
        1. False
        2. True
        3. True
        4. False

In [14]:
quiz_response = chain.invoke({"num_questions": 2, "quiz_type": "Open-ended", "quiz_context": "Python programming"})

In [15]:
Markdown(quiz_response)

- Questions:
    1. What is Python programming?
    2. How is Python different from other programming languages?
    
- Answers:
    1. Python programming is a high-level, interpreted, and general-purpose programming language known for its simplicity and readability.
    2. Python is different from other programming languages due to its easy syntax, dynamic typing, and extensive standard library.